# Image (.hdr, .mhd, etc...) manipulation for GATE

## Contents:
* Reading images with NumPy
* Reading images with SimpleITK
* Converting Dicom image serie to MetaImages (.mhd/.raw)
* Merging images

# Reading images

## Reading any binary image
PET mouse data from https://neuroimage.usc.edu/neuro/Digimouse_Download in Analyze (.hdr / .img) format.

In [ ]:
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import seaborn as sns
import numpy.ma as ma
import SimpleITK as sitk
import pydicom as dcm
import glob
# %matplotlib inline

In [ ]:
file = 'data/mouse_PET.img'
data_type = np.float32
dimension = [128, 128, 129]

# read the binary file
image = np.fromfile(file, dtype=data_type)
image = image.reshape(dimension,order='F')

print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
# cmap = sns.color_palette("mako_r", as_cmap=True)
# cmap = 'bone_r'
cmap = 'cividis'
# cmap = sns.color_palette("icefire", as_cmap=True)
# cmap = sns.color_palette("coolwarm", as_cmap=True)

plt.figure()
plt.imshow(ma.masked_less_equal(image[63,:,:], 0), cmap=cmap)
# plt.imshow(ma.masked_less_equal(image[63,:,:], 0), cmap=cmap, norm=mplc.LogNorm(1, vmax=image.max()))
# plt.imshow(image[63,:,:], cmap=plt.cm.jet)
plt.colorbar()
plt.show()

In [ ]:
def plot_image(myslice):
    plt.figure()
    plt.imshow(image[myslice,:,:], cmap=cmap, vmin=image.min(), vmax=image.max())
    plt.colorbar()
    plt.show()
    return;

ipw.interact(plot_image, myslice=(0, image.shape[0]-1));

In [ ]:
# save the binary image
image.tofile('output/mouse_PET_modified.img')

## Reading Meta images (.mhd / .raw)
Easy image reading with the library SimpleITK

In [ ]:
# 1) Read the image
itkimage = sitk.ReadImage('data/patient_CT.mhd')
image = sitk.GetArrayFromImage(itkimage)

print ('itkimage.GetSize() :' , itkimage.GetSize())
print ('itkimage.GetSpacing() :' , itkimage.GetSpacing())

print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
plt.figure()
plt.imshow(image[140,:,:], cmap='bone')
plt.colorbar()
plt.show()

In [ ]:
def plot_image(tran, coro, sagi):
    mini = image.min()
    maxi = image.max()
    cmap = 'bone'
    
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(image[tran, :, :], origin='lower', cmap=cmap, vmin=mini, vmax=maxi)
    plt.colorbar()
    
    plt.subplot(1, 3, 2)
    plt.imshow(image[:, coro, :], origin='lower', cmap=cmap, vmin=mini, vmax=maxi)
    plt.colorbar()
    
    plt.subplot(1, 3, 3)
    plt.imshow(image[:, :, sagi], origin='lower', cmap=cmap, vmin=mini, vmax=maxi)
    plt.colorbar()
    plt.show()
    return;

ipw.interact(plot_image, tran=(0, image.shape[0]-1), coro=(0, image.shape[1]-1), sagi=(0, image.shape[2]-1));

In [ ]:
# save the image
output = sitk.GetImageFromArray(image)
sitk.WriteImage(output,'output/patient_CT.mhd')

## Exercise
* modify the image data so that the slices of the CT (220) correspond to the last dimension
* crop the image in the sagittal plane to 25:85
* save the modified image

In [ ]:
# complete here ...

## Reading Dicom images (.dcm)
Easy image reading with the library SimpleITK

In [ ]:
# 1) Read the image
itkimage = sitk.ReadImage('data/patient_SPECT.dcm')
image = sitk.GetArrayFromImage(itkimage)

print ('itkimage.GetSpacing() :' , itkimage.GetSpacing())
print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
plt.figure()
plt.imshow(image[140, :, :], cmap='bone')
plt.colorbar()
plt.show()

## Reading Dicom images (.dcm)
Easy image reading with the library PyDicom

In [ ]:
# open dicom file (works for all .dcm file)
File = dcm.read_file('data/patient_SPECT.dcm')
# Acces to image pixel data 
image = File.pixel_array

print(type(image))
print('Pixel spacing :', File.PixelSpacing)
print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
plt.figure()
plt.imshow(image[140, :, :], cmap='bone')
plt.colorbar()
plt.show()

# Converting DICOM to MetaImage (.mhd/.raw)

In [ ]:
input_dir = 'convert/dicom/'
output_dir = 'convert/mhd/'

series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(input_dir)

if not series_IDs:
    print('No series in directory ' + input_dir)

for series in series_IDs:
    filename = series.split('.')[-1]
    sitk.WriteImage(sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(input_dir, series)),
                    output_dir+filename+'.mhd')
    print('Dicom serie converted to:', output_dir+filename+'.mhd')

## Exercise
* place a copy of the data/patient_CT_dcm/*.dcm images in the folder convert/dicom/  
* convert the Dicom images serie to a single .mhd image using the cell above
* read the converted image and display an axial slice of it

In [ ]:
# complete here ...

# Merging images

## Merge GATE output DoseActor .mhd of splitted simulations

In [ ]:
# directory of your simulation splitted outputs
output_dir = 'output/'

# loop over the .mhd outputs
first_image = True
for file in glob.glob(output_dir+'*Dose.mhd'):
#    if '_merged' not in file:
        print ('Reading file:',file)
        # 1) Read the image
        itkimage = sitk.ReadImage(file)
        # 2) Sum the image values
        if first_image:
            image_merged = sitk.GetArrayFromImage(itkimage)
            first_image = False
        else: 
            image_merged += sitk.GetArrayFromImage(itkimage)
        #print ('Sum of all voxels:',image_merged.max())

# 3) Write the sum of all images
itkimage_merged = sitk.GetImageFromArray(image_merged)
itkimage_merged.CopyInformation(itkimage)
sitk.WriteImage(itkimage_merged, output_dir+'output-Dose_merged.mhd')

print ('Merged file created:', output_dir+'output-Dose_merged.mhd')
print ('Dimension : ' , itkimage_merged.GetSize())
print ('Pixels Size :' , itkimage_merged.GetSpacing())

---

## Files opening summary

- Text files:
    - Base Python (open)
    - Numpy (loadtxt) if numpy array file (.txt, .dat, etc)
    - Pandas (read_csv) if CSV file (.csv, .tsv)
- Binary files:
    - Base Python (open)
    - Numpy (load) if numpy array or structured array file (.npy)
    - Numpy (fromfile) if raw binary (.raw, .img, etc)
- Excel files:
    - Pandas (.xls, .ods)
- MHD files:
    - SimpleITK
- Dicom files:
    - SimpleITK
    - pydicom
- Root files:
    - uproot
    
**In any cases, output should extracted to a numpy array or pandas dataframe**